In [2]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('chinook.db')
customers_df = pd.read_sql_query("SELECT * FROM customers", conn)
invoices_df = pd.read_sql_query("SELECT * FROM invoices", conn)
merged_df = pd.merge(customers_df, invoices_df, on='CustomerId', how='inner')
invoice_counts = merged_df.groupby(['CustomerId', 'FirstName', 'LastName']).size().reset_index(name='InvoiceCount')
print(invoice_counts.head())

In [ ]:

movie_df = pd.read_csv('movie.csv')
df1 = movie_df[['director_name', 'color']].dropna()
df2 = movie_df[['director_name', 'num_critic_for_reviews']].dropna()
df1 = df1.drop_duplicates(subset='director_name')
df2 = df2.drop_duplicates(subset='director_name')
left_join_df = pd.merge(df1, df2, on='director_name', how='left')
full_outer_join_df = pd.merge(df1, df2, on='director_name', how='outer')
print("Left Join Row Count:", len(left_join_df))
print("Full Outer Join Row Count:", len(full_outer_join_df))


In [ ]:

titanic_df = pd.read_excel('titanic.xlsx')
grouped_titanic = titanic_df.groupby('Pclass').agg({
    'Age': 'mean',
    'Fare': 'sum',
    'PassengerId': 'count'
}).rename(columns={
    'Age': 'AverageAge',
    'Fare': 'TotalFare',
    'PassengerId': 'PassengerCount'
}).reset_index()

print(grouped_titanic)


In [ ]:

flights_df = pd.read_parquet('flights.parquet')
grouped_flights = flights_df.groupby(['Year', 'Month']).agg({
    'FlightNum': 'count',
    'ArrDelay': 'mean',
    'DepDelay': 'max'
}).rename(columns={
    'FlightNum': 'TotalFlights',
    'ArrDelay': 'AverageArrDelay',
    'DepDelay': 'MaxDepDelay'
}).reset_index()

print(grouped_flights.head())


In [ ]:
titanic_df = pd.read_excel('titanic.xlsx')
def classify_age(age):
    if pd.isnull(age):
        return 'Unknown'
    return 'Child' if age < 18 else 'Adult'
titanic_df['Age_Group'] = titanic_df['Age'].apply(classify_age)

print(titanic_df[['Age', 'Age_Group']].head())


In [ ]:
employee_df = pd.read_csv('employee.csv')
employee_df['Normalized_Salary'] = employee_df.groupby('Department')['Salary'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

print(employee_df[['Department', 'Salary', 'Normalized_Salary']].head())


In [ ]:
movie_df = pd.read_csv('movie.csv')
def classify_duration(duration):
    if pd.isnull(duration):
        return 'Unknown'
    elif duration < 60:
        return 'Short'
    elif duration <= 120:
        return 'Medium'
    else:
        return 'Long'
movie_df['Duration_Label'] = movie_df['duration'].apply(classify_duration)

print(movie_df[['duration', 'Duration_Label']].head())


In [ ]:
titanic_df = pd.read_excel('titanic.xlsx')
def filter_survivors(df):
    return df[df['Survived'] == 1]

def fill_missing_age(df):
    return df.assign(Age=df['Age'].fillna(df['Age'].mean()))

def create_fare_per_age(df):
    return df.assign(Fare_Per_Age=df['Fare'] / df['Age'])
titanic_pipeline = (
    titanic_df
    .pipe(filter_survivors)
    .pipe(fill_missing_age)
    .pipe(create_fare_per_age)
)

print(titanic_pipeline[['Survived', 'Age', 'Fare', 'Fare_Per_Age']].head())


In [ ]:
flights_df = pd.read_parquet('flights.parquet')
def filter_delayed_flights(df):
    return df[df['DepDelay'] > 30]

def add_delay_per_hour(df):
    return df.assign(Delay_Per_Hour=df['DepDelay'] / df['AirTime'])
flights_pipeline = (
    flights_df
    .pipe(filter_delayed_flights)
    .pipe(add_delay_per_hour)
)

print(flights_pipeline[['DepDelay', 'AirTime', 'Delay_Per_Hour']].head())
